Connected to Python 3.13.5

In [1]:
# filepath: c:\Users\kenny\workspace\ContactEstimator\motion_equation.ipynb
import sympy as sp
import numpy as np
from sympy import symbols, Function, Matrix, cos, sin, diff, simplify, sqrt

print("Quadruped Robot Full-Body Dynamics Derivation")
print("=" * 50)

# Step 1: Define all symbolic variables
print("Step 1: Defining symbolic variables...")

# Time variable
t = symbols('t')

# Physical and geometric constants
m_b, m_l = symbols('m_b m_l', positive=True)  # Base mass, leg mass
I_bxx, I_byy = symbols('I_bxx I_byy', positive=True)  # Base inertias
X_offset = symbols('X_offset', positive=True)  # X_offset of hip from base center
Y_offset = symbols('Y_offset', positive=True)  # Y_offset of hip from base center
g = symbols('g', positive=True)  # Gravity acceleration

# Time-dependent leg rotational inertias (for simplified model)
I_c_lf = Function('I_c_lf')(t)  # Left-front leg inertia
I_c_rf = Function('I_c_rf')(t)  # Right-front leg inertia  
I_c_rh = Function('I_c_rh')(t)  # Right-hind leg inertia
I_c_lh = Function('I_c_lh')(t)  # Left-hind leg inertia

# Generalized coordinates as functions of time
x = Function('x')(t)
z = Function('z')(t)
phi = Function('phi')(t)  # Roll
psi = Function('psi')(t)  # Pitch

# Leg coordinates: beta (rotation angle), Rm (extension length)
beta_lf = Function('beta_lf')(t)
Rm_lf = Function('Rm_lf')(t)
beta_rf = Function('beta_rf')(t)
Rm_rf = Function('Rm_rf')(t)
beta_rh = Function('beta_rh')(t)
Rm_rh = Function('Rm_rh')(t)
beta_lh = Function('beta_lh')(t)
Rm_lh = Function('Rm_lh')(t)

# Generalized coordinates vector (12x1)
q = Matrix([x, z, phi, psi, beta_lf, Rm_lf, beta_rf, Rm_rf, beta_rh, Rm_rh, beta_lh, Rm_lh])

# Generalized velocities and accelerations
q_dot = Matrix([diff(qi, t) for qi in q])
q_ddot = Matrix([diff(qi, t, 2) for qi in q])

print(f"Defined {len(q)} generalized coordinates")

# Define actuator forces/torques as functions of time
print("Defining actuator forces/torques...")
tau_lf_beta = Function('tau_lf_beta')(t)
F_lf_Rm = Function('F_lf_Rm')(t)
tau_rf_beta = Function('tau_rf_beta')(t)
F_rf_Rm = Function('F_rf_Rm')(t)
tau_rh_beta = Function('tau_rh_beta')(t)
F_rh_Rm = Function('F_rh_Rm')(t)
tau_lh_beta = Function('tau_lh_beta')(t)
F_lh_Rm = Function('F_lh_Rm')(t)

# Actuator torque vector (8x1)
tau = Matrix([
    tau_lf_beta, F_lf_Rm,
    tau_rf_beta, F_rf_Rm,
    tau_rh_beta, F_rh_Rm,
    tau_lh_beta, F_lh_Rm
])

print("Actuator torque vector 'tau' created.")
# print(tau.shape)

Quadruped Robot Full-Body Dynamics Derivation
Step 1: Defining symbolic variables...
Defined 12 generalized coordinates
Defining actuator forces/torques...
Actuator torque vector 'tau' created.


In [2]:
q

Matrix([
[      x(t)],
[      z(t)],
[    phi(t)],
[    psi(t)],
[beta_lf(t)],
[  Rm_lf(t)],
[beta_rf(t)],
[  Rm_rf(t)],
[beta_rh(t)],
[  Rm_rh(t)],
[beta_lh(t)],
[  Rm_lh(t)]])

In [3]:
# filepath: c:\Users\kenny\workspace\ContactEstimator\motion_equation.ipynb
# Step 2: Kinematic analysis and total kinetic energy calculation
print("Step 2: Computing kinematic analysis and total kinetic energy...")

# Base kinetic energy
K_base = sp.Rational(1,2) * m_b * (diff(x,t)**2 + diff(z,t)**2) + \
         sp.Rational(1,2) * I_bxx * diff(phi,t)**2 + \
         sp.Rational(1,2) * I_byy * diff(psi,t)**2

print("Base kinetic energy computed")

# Base rotation matrix R_b = R_x(phi) * R_y(psi)
R_x_phi = Matrix([
    [1, 0, 0],
    [0, cos(phi), -sin(phi)],
    [0, sin(phi), cos(phi)]
])

R_y_psi = Matrix([
    [cos(psi), 0, sin(psi)],
    [0, 1, 0],
    [-sin(psi), 0, cos(psi)]
])

R_b = R_x_phi * R_y_psi

# sp.pprint(R_b)

# Mounting point vectors (from base center to leg attachment points)
r_H = {
    'lf': Matrix([X_offset, Y_offset, 0]),
    'rf': Matrix([X_offset, -Y_offset, 0]),
    'rh': Matrix([-X_offset, -Y_offset, 0]),
    'lh': Matrix([-X_offset, Y_offset, 0])
}

# Leg parameters dictionary with time-dependent inertias
legs = {
    'lf': {'beta': beta_lf, 'Rm': Rm_lf, 'I_c': I_c_lf},
    'rf': {'beta': beta_rf, 'Rm': Rm_rf, 'I_c': I_c_rf},
    'rh': {'beta': beta_rh, 'Rm': Rm_rh, 'I_c': I_c_rh},
    'lh': {'beta': beta_lh, 'Rm': Rm_lh, 'I_c': I_c_lh}
}

K_legs = 0
leg_positions = {}

for leg_name, leg_params in legs.items():
    print(f"Processing leg: {leg_name}")
    
    beta = leg_params['beta']
    Rm = leg_params['Rm']
    I_c = leg_params['I_c']  # Now time-dependent function
    
    # Leg center of mass relative to mounting point (in base frame)
    p_m_rel = Matrix([-Rm * sin(beta), 0, -Rm * cos(beta)]) 

    # Absolute position of leg center of mass
    base_position = Matrix([x, 0, z])
    p_m_abs = base_position + R_b * (r_H[leg_name] + p_m_rel)
    # print(r_H[leg_name])
    # print(f"  Leg center of mass position: {p_m_abs}")
    
    # Store for potential energy calculation
    leg_positions[leg_name] = p_m_abs
    
    # Absolute velocity of leg center of mass
    v_m_abs = Matrix([diff(p_m_abs[i], t) for i in range(3)])
    
    # Translational kinetic energy of this leg
    K_leg_trans = sp.Rational(1,2) * m_l * (v_m_abs.dot(v_m_abs))
    
    # Rotational kinetic energy of this leg around beta axis (with time-dependent inertia)
    K_leg_rot = sp.Rational(1,2) * I_c * (diff(beta, t)**2 + diff(psi, t)**2)
    
    # Total kinetic energy for this leg
    K_leg = K_leg_trans + K_leg_rot
    K_legs += K_leg
    
    print(f"  Translational KE:")
    print(sp.latex(sp.simplify(v_m_abs)))
    print(f"  Rotational KE:")
    print(sp.latex(K_leg_rot))

print("Leg kinetic energies (translational + rotational) computed")
print("Note: Inertias I_c_xx(t) are time-dependent for simplified model")

# Total kinetic energy
K_total = K_base + K_legs
print("Total kinetic energy computed")

Step 2: Computing kinematic analysis and total kinetic energy...
Base kinetic energy computed
Processing leg: lf
  Translational KE:
\left[\begin{matrix}- X_{offset} \sin{\left(\psi{\left(t \right)} \right)} \frac{d}{d t} \psi{\left(t \right)} - \operatorname{Rm}_{lf}{\left(t \right)} \cos{\left(\beta_{lf}{\left(t \right)} + \psi{\left(t \right)} \right)} \frac{d}{d t} \beta_{lf}{\left(t \right)} - \operatorname{Rm}_{lf}{\left(t \right)} \cos{\left(\beta_{lf}{\left(t \right)} + \psi{\left(t \right)} \right)} \frac{d}{d t} \psi{\left(t \right)} - \sin{\left(\beta_{lf}{\left(t \right)} + \psi{\left(t \right)} \right)} \frac{d}{d t} \operatorname{Rm}_{lf}{\left(t \right)} + \frac{d}{d t} x{\left(t \right)}\\X_{offset} \sin{\left(\phi{\left(t \right)} \right)} \cos{\left(\psi{\left(t \right)} \right)} \frac{d}{d t} \psi{\left(t \right)} + X_{offset} \sin{\left(\psi{\left(t \right)} \right)} \cos{\left(\phi{\left(t \right)} \right)} \frac{d}{d t} \phi{\left(t \right)} - Y_{offset} \sin{\lef

In [4]:
K_total_simplified = simplify(K_total)
print("Total kinetic energy simplified:")

Total kinetic energy simplified:


In [5]:
K_total_simplified

I_bxx*Derivative(phi(t), t)**2/2 + I_byy*Derivative(psi(t), t)**2/2 + 2*X_offset**2*m_l*sin(psi(t))**2*Derivative(phi(t), t)**2 + 2*X_offset**2*m_l*Derivative(psi(t), t)**2 - X_offset*m_l*Rm_lf(t)*sin(beta_lf(t))*Derivative(beta_lf(t), t)*Derivative(psi(t), t) - X_offset*m_l*Rm_lf(t)*sin(beta_lf(t))*Derivative(psi(t), t)**2 + X_offset*m_l*Rm_lf(t)*sin(psi(t))*cos(beta_lf(t) + psi(t))*Derivative(phi(t), t)**2 + X_offset*m_l*Rm_lh(t)*sin(beta_lh(t))*Derivative(beta_lh(t), t)*Derivative(psi(t), t) + X_offset*m_l*Rm_lh(t)*sin(beta_lh(t))*Derivative(psi(t), t)**2 - X_offset*m_l*Rm_lh(t)*sin(psi(t))*cos(beta_lh(t) + psi(t))*Derivative(phi(t), t)**2 - X_offset*m_l*Rm_rf(t)*sin(beta_rf(t))*Derivative(beta_rf(t), t)*Derivative(psi(t), t) - X_offset*m_l*Rm_rf(t)*sin(beta_rf(t))*Derivative(psi(t), t)**2 + X_offset*m_l*Rm_rf(t)*sin(psi(t))*cos(beta_rf(t) + psi(t))*Derivative(phi(t), t)**2 + X_offset*m_l*Rm_rh(t)*sin(beta_rh(t))*Derivative(beta_rh(t), t)*Derivative(psi(t), t) + X_offset*m_l*Rm_rh(t

In [6]:
# Step 3: Calculate total potential energy
print("Step 3: Computing total potential energy...")

# Base potential energy
P_base = m_b * g * z

# Legs potential energy
P_legs = 0
for leg_name, p_m_abs in leg_positions.items():
    P_leg = m_l * g * p_m_abs[2]  # Z component
    P_legs += P_leg

# Total potential energy
P_total = P_base + P_legs
print("Total potential energy computed")

Step 3: Computing total potential energy...
Total potential energy computed


In [7]:
P_total_simplified = simplify(P_total)

In [8]:
P_total_simplified

g*(m_b*z(t) - m_l*Rm_lf(t)*cos(beta_lf(t) + psi(t))*cos(phi(t)) - m_l*Rm_lh(t)*cos(beta_lh(t) + psi(t))*cos(phi(t)) - m_l*Rm_rf(t)*cos(beta_rf(t) + psi(t))*cos(phi(t)) - m_l*Rm_rh(t)*cos(beta_rh(t) + psi(t))*cos(phi(t)) + 4*m_l*z(t))

In [9]:
print("Leg positions (absolute coordinates):")
print("=" * 40)

for leg_name, p_m_abs in leg_positions.items():
    print(f"\nLeg {leg_name.upper()}:")
    print(f"  x-coordinate: {p_m_abs[0]}")
    print(f"  y-coordinate: {p_m_abs[1]}")
    print(f"  z-coordinate: {p_m_abs[2]}")

Leg positions (absolute coordinates):

Leg LF:
  x-coordinate: (X_offset - Rm_lf(t)*sin(beta_lf(t)))*cos(psi(t)) - Rm_lf(t)*sin(psi(t))*cos(beta_lf(t)) + x(t)
  y-coordinate: Y_offset*cos(phi(t)) + (X_offset - Rm_lf(t)*sin(beta_lf(t)))*sin(phi(t))*sin(psi(t)) + Rm_lf(t)*sin(phi(t))*cos(beta_lf(t))*cos(psi(t))
  z-coordinate: Y_offset*sin(phi(t)) - (X_offset - Rm_lf(t)*sin(beta_lf(t)))*sin(psi(t))*cos(phi(t)) - Rm_lf(t)*cos(beta_lf(t))*cos(phi(t))*cos(psi(t)) + z(t)

Leg RF:
  x-coordinate: (X_offset - Rm_rf(t)*sin(beta_rf(t)))*cos(psi(t)) - Rm_rf(t)*sin(psi(t))*cos(beta_rf(t)) + x(t)
  y-coordinate: -Y_offset*cos(phi(t)) + (X_offset - Rm_rf(t)*sin(beta_rf(t)))*sin(phi(t))*sin(psi(t)) + Rm_rf(t)*sin(phi(t))*cos(beta_rf(t))*cos(psi(t))
  z-coordinate: -Y_offset*sin(phi(t)) - (X_offset - Rm_rf(t)*sin(beta_rf(t)))*sin(psi(t))*cos(phi(t)) - Rm_rf(t)*cos(beta_rf(t))*cos(phi(t))*cos(psi(t)) + z(t)

Leg RH:
  x-coordinate: (-X_offset - Rm_rh(t)*sin(beta_rh(t)))*cos(psi(t)) - Rm_rh(t)*sin(psi(t

In [10]:
print("Leg velocity (absolute coordinates):")

for leg_name, p_m_abs in leg_positions.items():
    v_m_abs = Matrix([diff(p_m_abs[i], t) for i in range(3)])
    print(f"\nLeg {leg_name.upper()}:")
    print(f"  x-velocity: {v_m_abs[0]}")
    print(f"  y-velocity: {v_m_abs[1]}")
    print(f"  z-velocity: {v_m_abs[2]}")

Leg velocity (absolute coordinates):

Leg LF:
  x-velocity: -(X_offset - Rm_lf(t)*sin(beta_lf(t)))*sin(psi(t))*Derivative(psi(t), t) + (-Rm_lf(t)*cos(beta_lf(t))*Derivative(beta_lf(t), t) - sin(beta_lf(t))*Derivative(Rm_lf(t), t))*cos(psi(t)) + Rm_lf(t)*sin(beta_lf(t))*sin(psi(t))*Derivative(beta_lf(t), t) - Rm_lf(t)*cos(beta_lf(t))*cos(psi(t))*Derivative(psi(t), t) - sin(psi(t))*cos(beta_lf(t))*Derivative(Rm_lf(t), t) + Derivative(x(t), t)
  y-velocity: -Y_offset*sin(phi(t))*Derivative(phi(t), t) + (X_offset - Rm_lf(t)*sin(beta_lf(t)))*sin(phi(t))*cos(psi(t))*Derivative(psi(t), t) + (X_offset - Rm_lf(t)*sin(beta_lf(t)))*sin(psi(t))*cos(phi(t))*Derivative(phi(t), t) + (-Rm_lf(t)*cos(beta_lf(t))*Derivative(beta_lf(t), t) - sin(beta_lf(t))*Derivative(Rm_lf(t), t))*sin(phi(t))*sin(psi(t)) - Rm_lf(t)*sin(beta_lf(t))*sin(phi(t))*cos(psi(t))*Derivative(beta_lf(t), t) - Rm_lf(t)*sin(phi(t))*sin(psi(t))*cos(beta_lf(t))*Derivative(psi(t), t) + Rm_lf(t)*cos(beta_lf(t))*cos(phi(t))*cos(psi(t))*De

In [11]:
# Step 4: Apply Lagrange-Euler equations
print("Step 4: Applying Lagrange-Euler equations...")

# Lagrangian
L = K_total - P_total

# Calculate equations of motion
EOM_list = []
print("Computing equations of motion for each generalized coordinate...")

for i, qi in enumerate(q):
    print(f"Processing coordinate {i+1}/12: {qi}")
    qi_dot = diff(qi, t)
    
    # Lagrange-Euler equation: d/dt(∂L/∂q̇_i) - ∂L/∂q_i = 0
    dL_dqi_dot = diff(L, qi_dot)
    d_dt_dL_dqi_dot = diff(dL_dqi_dot, t)
    dL_dqi = diff(L, qi)
    
    EOM_i = d_dt_dL_dqi_dot - dL_dqi
    EOM_list.append(EOM_i)

# Create EOM vector
EOM_vec = Matrix(EOM_list)
print("Equations of motion computed")

Step 4: Applying Lagrange-Euler equations...
Computing equations of motion for each generalized coordinate...
Processing coordinate 1/12: x(t)
Processing coordinate 2/12: z(t)
Processing coordinate 3/12: phi(t)
Processing coordinate 4/12: psi(t)
Processing coordinate 5/12: beta_lf(t)
Processing coordinate 6/12: Rm_lf(t)
Processing coordinate 7/12: beta_rf(t)
Processing coordinate 8/12: Rm_rf(t)
Processing coordinate 9/12: beta_rh(t)
Processing coordinate 10/12: Rm_rh(t)
Processing coordinate 11/12: beta_lh(t)
Processing coordinate 12/12: Rm_lh(t)
Equations of motion computed


In [12]:
# Step 4.5: Define the Actuator Selection Matrix S^T
print("Step 4.5: Defining actuator selection matrix S^T...")

# q is a 12x1 vector. tau is an 8x1 vector. S^T must be a 12x8 matrix.
# It maps the 8 actuator forces to the 12 generalized coordinates.
# The unactuated coordinates (x, z, phi, psi) will have zero rows.

S_T = sp.zeros(12, 8)

# Map beta_lf and Rm_lf torques (tau columns 0, 1) to q rows 4, 5
S_T[4, 0] = 1  # tau_lf_beta -> beta_lf
S_T[5, 1] = 1  # tau_lf_Rm   -> Rm_lf

# Map beta_rf and Rm_rf torques (tau columns 2, 3) to q rows 6, 7
S_T[6, 2] = 1  # tau_rf_beta -> beta_rf
S_T[7, 3] = 1  # tau_rf_Rm   -> Rm_rf

# Map beta_rh and Rm_rh torques (tau columns 4, 5) to q rows 8, 9
S_T[8, 4] = 1  # tau_rh_beta -> beta_rh
S_T[9, 5] = 1  # tau_rh_Rm   -> Rm_rh

# Map beta_lh and Rm_lh torques (tau columns 6, 7) to q rows 10, 11
S_T[10, 6] = 1 # tau_lh_beta -> beta_lh
S_T[11, 7] = 1 # tau_lh_Rm   -> Rm_lh

S = S_T.T

print("Selection matrix S_T created with shape:", S_T.shape)

Step 4.5: Defining actuator selection matrix S^T...
Selection matrix S_T created with shape: (12, 8)


In [13]:
# Step 5: Extract M, C, G matrices/vectors using proper Christoffel symbols
print("Step 5: Extracting M, C, G matrices/vectors...")

# Mass matrix M(q): coefficient matrix of q_ddot
print("Computing mass matrix M...")
M = EOM_vec.jacobian(q_ddot)

# Gravity vector G(q): EOM when all velocities and accelerations are zero
print("Computing gravity vector G...")
subs_zero_vel_accel = {}
for qi_dot in q_dot:
    subs_zero_vel_accel[qi_dot] = 0
for qi_ddot in q_ddot:
    subs_zero_vel_accel[qi_ddot] = 0

G = EOM_vec.subs(subs_zero_vel_accel)

# Compute Coriolis matrix C using Christoffel symbols method
print("Computing Coriolis matrix C using Christoffel symbols...")
print("This computation may take some time...")

n = len(q)  # Number of generalized coordinates (12)

# Step 1: Compute time derivative of mass matrix M_dot
print("Computing M_dot (time derivative of mass matrix)...")
M_dot = sp.zeros(n, n)
for i in range(n):
    for j in range(n):
        # Chain rule: dM_ij/dt = sum_k (dM_ij/dq_k * dq_k/dt)
        M_dot_ij = 0
        for k in range(n):
            dM_ij_dqk = diff(M[i, j], q[k])
            M_dot_ij += dM_ij_dqk * q_dot[k]
        M_dot[i, j] = M_dot_ij

print("M_dot computed")

# Step 2: Compute Christoffel symbols c_ijk
print("Computing Christoffel symbols c_ijk...")
# Use dictionary to store 3D tensor c[i][j][k]
christoffel = {}

for i in range(n):
    for j in range(n):
        for k in range(n):
            # c_ijk = 1/2 * (dM_kj/dq_i + dM_ki/dq_j - dM_ij/dq_k)
            dM_kj_dqi = diff(M[k, j], q[i])
            dM_ki_dqj = diff(M[k, i], q[j])
            dM_ij_dqk = diff(M[i, j], q[k])
            
            christoffel[(i, j, k)] = sp.Rational(1, 2) * (dM_kj_dqi + dM_ki_dqj - dM_ij_dqk)

print("Christoffel symbols computed")

# Step 3: Compute Coriolis matrix C
print("Computing Coriolis matrix C...")
C_matrix = sp.zeros(n, n)

for k in range(n):
    for j in range(n):
        # C_kj = sum_i (c_ijk * q_dot_i)
        C_kj = 0
        for i in range(n):
            C_kj += christoffel[(i, j, k)] * q_dot[i]
        C_matrix[k, j] = C_kj

print("Coriolis matrix C computed")

# Compute Coriolis vector C_vec = C * q_dot
C_vec = C_matrix * q_dot

print("Coriolis vector C_vec computed")

D_matrix = sp.zeros(n, n)
# Add the time-dependent inertias to the diagonal of C_matrix
D_matrix[3, 3] += (sp.Derivative(I_c_lf, t) + sp.Derivative(I_c_rf, t) +
                   sp.Derivative(I_c_rh, t) + sp.Derivative(I_c_lh, t))  # Base inertia
D_matrix[4, 4] += sp.Derivative(I_c_lf, t)  # Left-front leg inertia
D_matrix[6, 6] += sp.Derivative(I_c_rf, t)  # Right-front leg inertia
D_matrix[8, 8] += sp.Derivative(I_c_rh, t)  # Right-hind leg inertia
D_matrix[10, 10] += sp.Derivative(I_c_lh, t)  # Left-hind leg inertia

print("D_matrix with time-dependent inertias computed")

# compute D_vec = D_matrix * q_dot
D_vec = D_matrix * q_dot

print("D_vec computed")

Step 5: Extracting M, C, G matrices/vectors...
Computing mass matrix M...
Computing gravity vector G...
Computing Coriolis matrix C using Christoffel symbols...
This computation may take some time...
Computing M_dot (time derivative of mass matrix)...
M_dot computed
Computing Christoffel symbols c_ijk...
Christoffel symbols computed
Computing Coriolis matrix C...
Coriolis matrix C computed
Coriolis vector C_vec computed
D_matrix with time-dependent inertias computed
D_vec computed


In [14]:
# Check M is symmetric and positive definite
print("Checking mass matrix M properties...")
print("=" * 50)

# 1. Check symmetry: M should equal M^T
print("1. Checking if M is symmetric...")
M_transpose = M.transpose()
M_minus_MT = M - M_transpose
M_symmetry_error = simplify(M_minus_MT)

is_symmetric = M_symmetry_error == sp.zeros(len(q), len(q))
print(f"M is symmetric: {is_symmetric}")

if not is_symmetric:
    print("Non-zero elements in M - M^T:")
    non_zero_count = 0
    for i in range(min(5, len(q))):  # Check first 5x5 submatrix
        for j in range(min(5, len(q))):
            if M_symmetry_error[i, j] != 0:
                print(f"  M[{i},{j}] - M[{j},{i}] = {M_symmetry_error[i, j]}")
                non_zero_count += 1
                if non_zero_count >= 10:  # Limit output
                    break
        if non_zero_count >= 10:
            break

# 2. Check positive definiteness
print("\n2. Checking if M is positive definite...")
print("For a kinetic energy-derived mass matrix, this should always be true")
print("if all masses and inertias are positive.")

# Check diagonal elements (necessary but not sufficient condition)
print("\nDiagonal elements of M:")
diagonal_elements = []
for i in range(len(q)):
    diag_elem = M[i, i]
    diagonal_elements.append(diag_elem)
    print(f"M[{i},{i}] = {diag_elem}")

# For kinetic energy matrices, positive definiteness is guaranteed if:
# - All masses (m_b, m_l) are positive ✓
# - All inertias (I_bxx, I_byy, I_c_*) are positive ✓
# - The kinetic energy expression is properly derived ✓

print("\n3. Physical verification of positive definiteness:")
print("Since M is derived from kinetic energy K = (1/2)q̇ᵀMq̇:")
print("- All physical parameters (masses, inertias) are defined as positive")
print("- The quadratic form represents kinetic energy, which is always ≥ 0")
print("- Therefore, M is positive semidefinite by construction")
print("- M is positive definite if the system has no rigid body modes")

# Check for any zero diagonal elements (which might indicate issues)
zero_diagonals = []
for i, elem in enumerate(diagonal_elements):
    if elem == 0:
        zero_diagonals.append(i)

if zero_diagonals:
    print(f"\nWARNING: Zero diagonal elements found at indices: {zero_diagonals}")
    print("This might indicate rigid body modes or modeling issues")
else:
    print("\n✓ All diagonal elements are non-zero expressions")

print("\nSUMMARY:")
print(f"✓ Symmetric: {is_symmetric}")
print("✓ Positive definite: True (by kinetic energy construction)")
print("✓ Mass matrix M satisfies required properties for robot dynamics")

Checking mass matrix M properties...
1. Checking if M is symmetric...
M is symmetric: True

2. Checking if M is positive definite...
For a kinetic energy-derived mass matrix, this should always be true
if all masses and inertias are positive.

Diagonal elements of M:
M[0,0] = m_b + 4*m_l
M[1,1] = m_b + 4*m_l
M[2,2] = I_bxx + m_l*((-2*Y_offset*sin(phi(t)) + 2*(-X_offset - Rm_lh(t)*sin(beta_lh(t)))*sin(psi(t))*cos(phi(t)) + 2*Rm_lh(t)*cos(beta_lh(t))*cos(phi(t))*cos(psi(t)))*(-Y_offset*sin(phi(t)) + (-X_offset - Rm_lh(t)*sin(beta_lh(t)))*sin(psi(t))*cos(phi(t)) + Rm_lh(t)*cos(beta_lh(t))*cos(phi(t))*cos(psi(t))) + (Y_offset*cos(phi(t)) + (-X_offset - Rm_lh(t)*sin(beta_lh(t)))*sin(phi(t))*sin(psi(t)) + Rm_lh(t)*sin(phi(t))*cos(beta_lh(t))*cos(psi(t)))*(2*Y_offset*cos(phi(t)) + 2*(-X_offset - Rm_lh(t)*sin(beta_lh(t)))*sin(phi(t))*sin(psi(t)) + 2*Rm_lh(t)*sin(phi(t))*cos(beta_lh(t))*cos(psi(t))))/2 + m_l*((-2*Y_offset*sin(phi(t)) + 2*(X_offset - Rm_lf(t)*sin(beta_lf(t)))*sin(psi(t))*cos(phi

In [15]:
# Verify skew-symmetric property of N = M_dot - 2C 
# the simplified model will not satisfy this property
print("\nVerifying skew-symmetric property...")
print("=" * 50)

# Compute N = M_dot - 2*C_matrix
N = M_dot - 2*C_matrix

print("Computing N = M_dot - 2*C...")

# Verify skew-symmetric property: N + N^T should be zero
N_transpose = N.transpose()
N_plus_NT = N + N_transpose

print("Checking if N + N^T = 0 (skew-symmetric property)...")

# Simplify the result
N_plus_NT_simplified = simplify(N_plus_NT)

# Check if it's a zero matrix
is_zero_matrix = N_plus_NT_simplified == sp.zeros(n, n)

print(f"\nSKEW-SYMMETRIC VERIFICATION RESULT:")
print(f"Matrix N dimensions: {N.shape}")
print(f"N + N^T is zero matrix: {is_zero_matrix}")

if is_zero_matrix:
    print("✓ VERIFICATION SUCCESSFUL: N = M_dot - 2C is skew-symmetric")
    print("✓ Coriolis matrix C satisfies the proper mathematical properties")
else:
    print("✗ VERIFICATION FAILED: N = M_dot - 2C is NOT skew-symmetric")
    print("This indicates an error in the Coriolis matrix calculation")
    
    # Show some non-zero elements if verification fails
    non_zero_count = 0
    for i in range(min(3, n)):  # Check first 3x3 submatrix
        for j in range(min(3, n)):
            if N_plus_NT_simplified[i, j] != 0:
                print(f"Non-zero element at [{i},{j}]: {N_plus_NT_simplified[i, j]}")
                non_zero_count += 1
                if non_zero_count >= 5:  # Limit output
                    break
        if non_zero_count >= 5:
            break
    
    if non_zero_count > 0:
        print(f"... and possibly more non-zero elements")

print("\nPhysical interpretation:")
print("The skew-symmetric property N = M_dot - 2C ensures that:")
print("1. Energy conservation in the absence of external forces")
print("2. Proper coupling between kinetic energy and Coriolis effects")
print("3. Passivity of the mechanical system")


Verifying skew-symmetric property...
Computing N = M_dot - 2*C...
Checking if N + N^T = 0 (skew-symmetric property)...

SKEW-SYMMETRIC VERIFICATION RESULT:
Matrix N dimensions: (12, 12)
N + N^T is zero matrix: True
✓ VERIFICATION SUCCESSFUL: N = M_dot - 2C is skew-symmetric
✓ Coriolis matrix C satisfies the proper mathematical properties

Physical interpretation:
The skew-symmetric property N = M_dot - 2C ensures that:
1. Energy conservation in the absence of external forces
2. Proper coupling between kinetic energy and Coriolis effects
3. Passivity of the mechanical system


In [16]:
# verify C_vetor is correct
print("\nStep 6: Verifying C_vector is correct")
print("=" * 50)
print()
C_vec_direct = EOM_vec - M * q_ddot - G
# Check if C_vec matches the direct computation
C_vec_error = (C_vec + D_vec) - C_vec_direct

print("C_vector is correct {}" if C_vec_error == sp.zeros(n, 1) else "C_vector is NOT correct")

C_matrix_simplified = simplify(C_matrix)



Step 6: Verifying C_vector is correct

C_vector is NOT correct


In [17]:
C_vec_simplified = simplify(C_vec)

In [18]:
C_vec_simplified

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [19]:
C_matrix_simplified

Matrix([
[0, 0,                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [20]:
C_vec_error_simplified = simplify(C_vec_error)

In [21]:
C_vec_error_simplified

Matrix([
[0],
[0],
[0],
[0],
[0],
[0],
[0],
[0],
[0],
[0],
[0],
[0]])

In [22]:
sp.latex(D_vec)

'\\left[\\begin{matrix}0\\\\0\\\\0\\\\\\left(\\frac{d}{d t} I_{c lf}{\\left(t \\right)} + \\frac{d}{d t} I_{c lh}{\\left(t \\right)} + \\frac{d}{d t} I_{c rf}{\\left(t \\right)} + \\frac{d}{d t} I_{c rh}{\\left(t \\right)}\\right) \\frac{d}{d t} \\psi{\\left(t \\right)}\\\\\\frac{d}{d t} I_{c lf}{\\left(t \\right)} \\frac{d}{d t} \\beta_{lf}{\\left(t \\right)}\\\\0\\\\\\frac{d}{d t} I_{c rf}{\\left(t \\right)} \\frac{d}{d t} \\beta_{rf}{\\left(t \\right)}\\\\0\\\\\\frac{d}{d t} I_{c rh}{\\left(t \\right)} \\frac{d}{d t} \\beta_{rh}{\\left(t \\right)}\\\\0\\\\\\frac{d}{d t} I_{c lh}{\\left(t \\right)} \\frac{d}{d t} \\beta_{lh}{\\left(t \\right)}\\\\0\\end{matrix}\\right]'

In [23]:
G_simplified = simplify(G)

In [24]:
G_simplified

Matrix([
[                                                                                                                                                                0],
[                                                                                                                                                  g*(m_b + 4*m_l)],
[g*m_l*(Rm_lf(t)*cos(beta_lf(t) + psi(t)) + Rm_lh(t)*cos(beta_lh(t) + psi(t)) + Rm_rf(t)*cos(beta_rf(t) + psi(t)) + Rm_rh(t)*cos(beta_rh(t) + psi(t)))*sin(phi(t))],
[g*m_l*(Rm_lf(t)*sin(beta_lf(t) + psi(t)) + Rm_lh(t)*sin(beta_lh(t) + psi(t)) + Rm_rf(t)*sin(beta_rf(t) + psi(t)) + Rm_rh(t)*sin(beta_rh(t) + psi(t)))*cos(phi(t))],
[                                                                                                              g*m_l*Rm_lf(t)*sin(beta_lf(t) + psi(t))*cos(phi(t))],
[                                                                                                                      -g*m_l*cos(beta_lf(t) + psi(t))*cos(phi(t))],
[

In [25]:
M_simplified = simplify(M)

In [26]:
M_simplified

Matrix([
[                                                                                                                                         m_b + 4*m_l,                                                                                                                                                               0,                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [27]:
D_matrix

Matrix([
[0, 0, 0,                                                                                                         0,                        0, 0,                        0, 0,                        0, 0,                        0, 0],
[0, 0, 0,                                                                                                         0,                        0, 0,                        0, 0,                        0, 0,                        0, 0],
[0, 0, 0,                                                                                                         0,                        0, 0,                        0, 0,                        0, 0,                        0, 0],
[0, 0, 0, Derivative(I_c_lf(t), t) + Derivative(I_c_lh(t), t) + Derivative(I_c_rf(t), t) + Derivative(I_c_rh(t), t),                        0, 0,                        0, 0,                        0, 0,                        0, 0],
[0, 0, 0,                                              

In [28]:
# Store all equations in a dictionary
equations_dict = {
    'M': M_simplified,
    'C': C_matrix_simplified,
    'D': D_matrix,
    'G': G_simplified,
    'S_T': S_T,
    'q': q,
    'q_dot': q_dot,
    'q_ddot': q_ddot,
    'tau': tau,
    'symbols': {
        'm_b': m_b, 'm_l': m_l, 'I_bxx': I_bxx, 'I_byy': I_byy, 'X_offset': X_offset, 'Y_offset': Y_offset, 'g': g, 't': t
    },
    'functions': {
        'x': x, 'z': z, 'phi': phi, 'psi': psi, 
        'beta_lf': beta_lf, 'Rm_lf': Rm_lf, 'I_c_lf': I_c_lf,
        'beta_rf': beta_rf, 'Rm_rf': Rm_rf, 'I_c_rf': I_c_rf,
        'beta_rh': beta_rh, 'Rm_rh': Rm_rh, 'I_c_rh': I_c_rh,
        'beta_lh': beta_lh, 'Rm_lh': Rm_lh, 'I_c_lh': I_c_lh,
        'tau_lf_beta': tau_lf_beta, 'F_lf_Rm': F_lf_Rm,
        'tau_rf_beta': tau_rf_beta, 'F_rf_Rm': F_rf_Rm,
        'tau_rh_beta': tau_rh_beta, 'F_rh_Rm': F_rh_Rm,
        'tau_lh_beta': tau_lh_beta, 'F_lh_Rm': F_lh_Rm
    }
}

In [29]:
import time
from pathlib import Path
import dill

# Create directory for saved equations
save_dir = Path(r'c:\Users\kenny\workspace\ContactEstimator\saved_equations')
save_dir.mkdir(exist_ok=True)
print("Saving with dill...")
start_time = time.time()

try:
    with open(save_dir / 'equations_dill.pkl', 'wb') as f:
        dill.dump(equations_dict, f)
    
    dill_save_time = time.time() - start_time
    print(f"✓ Dill save successful in {dill_save_time:.4f} seconds")
    print(f"File size: {(save_dir / 'equations_dill.pkl').stat().st_size / 1024:.2f} KB")
except Exception as e:
    print(f"✗ Dill save failed: {e}")

Saving with dill...
✓ Dill save successful in 0.0364 seconds
File size: 25.37 KB


In [30]:
S_T

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[1, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 0, 0, 1]])